# SETUP

In [ ]:
# run IPYNB that imports all the relevant packages and functions
%run ../ms_packages_and_functions.ipynb

In [ ]:
output_dir = pjoin(os.getcwd(), 'processed_data')
data_dir  = pjoin(os.getcwd(), 'raw_data')

In [ ]:
run_path = pjoin(output_dir,'RJ_null-metrics-reciprocity-004-cutoff')

if not os.path.exists(run_path):
    os.mkdir(run_path)
    print("created directory: {}".format(run_path))
else:
    print("All files will be saved in: {}".format(run_path))

In [ ]:
fig_dir = pjoin(run_path,'figures')

if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)
    print("created directory: {}".format(fig_dir))
else:
    print("All files will be saved in: {}".format(fig_dir))

# Null paths

In [ ]:
null_folders = [file for file in os.listdir(output_dir) if file.startswith('RJ_nulls-poisson-200ms-0')]
null_filepaths = [pjoin(output_dir,folder,file) for folder in null_folders for file in os.listdir(pjoin(output_dir,folder)) if file.startswith('poisson_FNs_')]

```
null_reciprocity = []
null_num = 0
for i,FN_path in enumerate(null_filepaths):
    start_time = time.time()
    try:
        null_FNs = loadPickle(FN_path)
        """
        process goes here
        """

        del null_FNs
        end_time = time.time()
        print(f'sim # {i}, elapsed (s) {end_time - start_time}')
        null_num += 1

    except:
        end_time = time.time()
        print(f'sim # {i}, broken! elapsed (s) {end_time - start_time}')
```

In [ ]:
print(f'{len(null_filepaths)} FOUND IN THE FOLLOWING FILEPATHS: \n{[x[-46:] for x in null_filepaths]}')

In [ ]:
trials = loadPickle('processed_data/rj031209_trials_by_cutoff.pkl')

# Reciprocity

In [ ]:
run_reciprocity = True

In [ ]:
if run_reciprocity:
    null_reciprocity = []
    null_num = 0
    num_processes = 8
    th = 75
    cutoff = 1.5
    for i,FN_path in enumerate(null_filepaths):
        # start_time = time.time()
        try:
            null_FNs = loadPickle(FN_path)
            dir_reciprocity = []
            for direction in range(8):
                trial_reciprocity = []
                for trial in range(len(null_FNs[0][direction][0])):
                    if trial in trials[cutoff]['trials'][direction]:
                        status(f"SIM #{i}: Reciprocity for direction {direction}, trial {trial}")

                        time_reciprocity = [weighted_reciprocity(thresholdNetwork(FN,th)) for FN in  null_FNs[0][direction][0][trial][0]]

                        # with Pool(num_processes) as p:
                        #     time_reciprocity = p.map(weighted_reciprocity,temp)
                        trial_reciprocity.append(time_reciprocity)

                dir_reciprocity.append(trial_reciprocity)
            null_reciprocity.append(dir_reciprocity)

            del null_FNs
            # end_time = time.time()
            # print(f'sim # {i}, elapsed (s) {end_time - start_time}')
            null_num += 1

        except Exception as e: 
            print(e)
            # end_time = time.time()
            print(f'sim # {i}, broken!')# elapsed (s) {end_time - start_time}')

    # savePickle(pjoin(run_path,f'null_reciprocity-{null_num}.pkl'),null_reciprocity)

    # r_null = [weighted_reciprocity(FN) for FN in  null_FNs[0][direction][0][trial][0]]
    savePickle(pjoin(run_path,f'null_reciprocity-{null_num}-p{th}-c{cutoff}.pkl'),null_reciprocity)

### Average

In [ ]:
# instruction
if run_reciprocity:
# average Reciprocity
    ave_null_rec = []

    for direction in range(8):
        trial_rec = []
        for trial in range(len(null_reciprocity[0][direction])):
            time_rec = []
            for time in range(len(null_reciprocity[0][direction][trial])):
                sim_rec = []
                for sim in range(len(null_reciprocity)):
                    sim_rec.append(null_reciprocity[sim][direction][trial][time])
                time_rec.append(np.mean(sim_rec))
            trial_rec.append(time_rec)
            # plt.plot(time_rec,'k',alpha=0.2)
        ave_null_rec.append(trial_rec)

    savePickle(pjoin(run_path,f'ave-null_reciprocity-{null_num}-p{th}-c{cutoff}.pkl'),ave_null_rec)

In [ ]:
FNs = loadPickle('processed_data/rj1031209_CenterOut_FN_10msBin_with500buffer_MIonly.pkl')
reciprocity_scores = []
for direction in range(8):
    trial_rec = []
    for trial in range(len(FNs[direction])):
        if trial in trials[cutoff]['trials'][direction]:

            r = [weighted_reciprocity(thresholdNetwork(FN,th)) for FN in FNs[direction][trial]]
            trial_rec.append(r)
    reciprocity_scores.append(trial_rec)

In [ ]:
if run_reciprocity:
    norm_reciprocity = []

    cmap = sns.husl_palette(8)
    dirs = [1,3,5,7]
    for direction in range(8):
        plt.figure()
        trial_rec = []

        for trial in range(len(null_reciprocity[0][direction])):
            time_rec = []
            for t in range(len(null_reciprocity[0][direction][trial])):
                nr = normalizeNetworkMetric(reciprocity_scores[direction][trial][t],ave_null_rec[direction][trial][t])
                time_rec.append(nr)
            trial_rec.append(time_rec)
            plt.plot(time_rec,c=cmap[direction],alpha=0.2)
            
        min_trial_length = min([len(x) for x in trial_rec])
        plt.plot(np.mean(np.array([x[:min_trial_length] for x in trial_rec]),axis=0),'k')
        
        norm_reciprocity.append(trial_rec)

    savePickle(pjoin(run_path,f'norm_reciprocity-{null_num}-p{th}.pkl'),norm_reciprocity)


In [ ]:
cmap = sns.color_palette('husl',8)
min_len = time2bin(1.5,lastBin=True,buffer=0.5)
x = [bin2time(x,lastBin=True) for x in np.arange(min_len)]
dirs = [1,3,5,7]

for i in range(8):

    plt.figure(figsize=(15,3))
    plt.subplot(131)
    r = reciprocity_scores[i]
    mean = np.mean(np.array([np.array(x[:min_len]) for x in r]),axis=0)
    err = np.std(np.array([np.array(x[:min_len]) for x in r]),axis=0)
    plt.plot([0,0],[min(mean),max(mean)], 'k--',linewidth=2)
    plt.plot(x,mean, color = cmap[i],linewidth=2)
    plt.title('Data')

    plt.subplot(132)
    r = ave_null_rec[i]
    mean = np.mean(np.array([np.array(x[:min_len]) for x in r]),axis=0)
    err = np.std(np.array([np.array(x[:min_len]) for x in r]),axis=0)
    plt.plot([0,0],[min(mean),max(mean)], 'k--',linewidth=2)
    plt.plot(x,mean,color = cmap[i],linewidth=2)
    plt.title(f'Null (n={null_num})')

    plt.subplot(133)
    r = norm_reciprocity[i]
    mean = np.mean(np.array([np.array(x[:min_len]) for x in r]),axis=0)
    err = np.std(np.array([np.array(x[:min_len]) for x in r]),axis=0)
    plt.plot([0,0],[min(mean),max(mean)], 'k--',linewidth=2)
    plt.plot(x,mean,color = cmap[i],linewidth=2)
    plt.title('Norm')
    sns.despine()

    plt.savefig(pjoin(fig_dir,f'reciprocity_{i}-p{th}.png'))
